In [1]:
# various tools

RR = RealField(2048)

from random import shuffle

img = sqrt(-1)

def frange(a,b,s):
    L=[]
    t = a
    while(t<b):
        L.append(t)
        t += s
    return L

def root_Hermite(b):
    b = RR(b)
    return RR(( (b/(2*pi*e))*((pi*b)^(1/b)))^(1/(2*(b-1))))

'''
@return the length of the vectors produced by the short vector sampler
@param d: dimension of the lattice
@param V: the volume of the lattice
@param b0: beta_0
@param b1: beta_1
'''
def short_vector_length(d, V, b0, b1):
    d = RR(d)
    V = RR(V)
    b0 = RR(b0)
    b1 = RR(b1)
    return RR( sqrt(4/3) * (V^(1/d)) * (root_Hermite(b1)^(b1-1)) * (root_Hermite(b0)^(d - b1)) )

def B(alpha, x):
    alpha = RR(alpha)
    x = RR(x)
    if x < 1e-100:
        return RR(1)
    return RR( RR(gamma(RR(alpha+RR(1))) * bessel_J(alpha, x)) / RR((x/RR(2))^alpha) )

In [ ]:
'''
@param T: threshold
@param N: number of short vectors sampled
@param alpha: the parameter of the centered binomial of the LWE oracle
@param q: the size of the field
@param m: the number of rows of the matrix A
@param nlat: the dimension of the lattice part
@param nfft: the length of the lossy source code
@param kfft: the dimension of the lossy source code
@param dlat: the radius of the ball in which the sampled short vectors are drawn uniformly
@param dlsc: the average decoding distance
@param slsc: the standard deviation of the decoding distance
'''

<b>Good Guess</b>

$$
\begin{array}{lcl}
P_\mathsf{good} & := & \mathbb{P}\left( F^{(\mathsf{lsc})}_{\mathbf{s}_{\mathsf{enu}}} \left( \mathbf{G}^{\top} \mathbf{s}_{\mathsf{fft}} \right) \geq T \right) \\
& \approx & 0.5
\end{array}
$$
if
$$
\frac{T}{N} = \frac{\exp\left( \tfrac{-\alpha(\pi \mu_{lsc}/q)^2}{1 + 2 \alpha (\pi \sigma_{lsc}/q)^2}\right)}{\sqrt{1 + 2 \alpha (\pi \sigma_{lsc}/q)^2}} \cdot \int_0^1 \beta_{sieve} \cdot t^{\beta_{sieve} - 1} \cdot e^{-\alpha\left( \frac{\pi d_{lat} t}{q}\right)^2} dt
$$

<b>Wrong Guess</b>

We assume that $(\widetilde{\mathbf{s}_{\mathsf{enu}}}, \widetilde{\mathbf{s}_{\mathsf{fft}}}) \neq (\mathbf{s}_{\mathsf{enu}}, \mathbf{s}_{\mathsf{fft}})$.


$$
\begin{array}{lcl}
P_\mathsf{wrong} & := & \mathbb{P}\left( F^{(\mathsf{lsc})}_{\widetilde{\mathbf{s}_{\mathsf{enu}}}} \left( \mathbf{G}^{\top} \widetilde{\mathbf{s}_{\mathsf{fft}}} \right) \geq T \right) \\
& \approx & \mathbb{P}\left( \mathcal{D} + \mathcal{N}\left(0, \sqrt{N/2}\right) \geq T \right) \\
& \approx &  \displaystyle{\int_{\mathbb{R}}} \displaystyle{\int_{\mathbb{R}^+}} \psi_\mathsf{lsc}(d_\mathsf{lsc})  \left( \min\left( 1 , \displaystyle{\int_{\mathcal{E}(T-t)}} \lambda(i) \mu(j) d(i,j) \right)\right) \cdot \tfrac{e^{-t^2/N}}{\sqrt{\pi N}}  dd_\mathsf{lsc} \; dt
\end{array}
$$
where
$$
\mathcal{E}(T-t) := \{ (i,j) \; : \;  N \cdot G(i,j) \geq T - t\}
$$
$$
G(i,j) := B\left( \tfrac{\beta_{\mathsf{sieve}}}{2}, \tfrac{2 \pi}{q} d_{\mathsf{lat}} i\right) \cdot B\left( \tfrac{n_{\mathsf{fft}}}{2} - 1, \tfrac{2 \pi}{q} d_{\mathsf{lsc}} j\right)
$$
and
$$
B(\alpha, x) := \frac{\Gamma(\alpha + 1) J_{\alpha}(x)}{(x/2)^{\alpha}}
$$
and 
$$
d_\mathsf{lsc} \sim \mathcal{N}(\mu_{\mathsf{lsc}}, \sigma_{\mathsf{lsc}})
$$
and
$$
\lambda(i) := \frac{2 \cdot \delta(\beta_{\mathsf{bkz}})^{\beta_{\mathsf{sieve}}(m + n_\mathsf{lat} - \beta_{\mathsf{sieve}})} \cdot \pi^{(\beta_{\mathsf{sieve}})/2} \cdot i^{\beta_{\mathsf{sieve}}-1}}{q^{\beta_{\mathsf{sieve}} \cdot \tfrac{m}{m + n_\mathsf{lat}}} \cdot \Gamma\left(\frac{\beta_{\mathsf{sieve}}}{2}\right)} 
$$ 
and
$$
\mu(j) := \frac{2 \cdot \pi^{n_{\mathsf{fft}}/2} \cdot j^{n_{\mathsf{fft}}-1}}{q^{k_{\mathsf{fft}}} \cdot \Gamma\left(\frac{n_{\mathsf{fft}}}{2}\right)} 
$$




In [2]:
import pickle

from estimator.estimator.cost import Cost
from estimator.estimator.lwe_parameters import LWEParameters
from estimator.estimator.lwe import Estimate
from estimator.estimator.reduction import delta as deltaf
from estimator.estimator.reduction import RC, ReductionCost
from estimator.estimator.conf import red_cost_model as red_cost_model_default
from estimator.estimator.util import local_minimum, early_abort_range
from estimator.estimator.io import Logging
from estimator.estimator.nd import NoiseDistribution
from estimator.estimator.schemes import (
    Kyber512,
    Kyber768,
    Kyber1024,
)

RR = RealField(2048)

def get_reduction_cost_model(nn):
    matzov_nns={
        "CN": "list_decoding-naive_classical",
        "CC": "list_decoding-classical",
    }
    if nn in matzov_nns:
        return RC.MATZOV.__class__(nn=matzov_nns[nn])
    elif nn == "C0":
        return RC.ADPS16
    else:
        raise Error("unknown cost model '{}'".format(nn))

def cost_sample(m, beta0, beta1, N, red_cost_model):
	rho, T, _, _ = red_cost_model.short_vectors(
		beta = beta0, N=N, d=m, sieve_dim=beta1
	)
	return rho, T
    
C_mul = RR(1024)
C_add = RR(160)

# survival function of D when dlsc is drawn as a normal with mean avg_dlsc and standard deviation sdv_dlsc (approximation)
def sv_D(T, N, q, m, alpha, nenu, nlat, nfft, kfft, beta0, beta1, dlat, avg_dlsc, sdv_dlsc):
    T = RR(T)
    N = RR(N)
    q = RR(q)
    alpha = RR(alpha)
    m = RR(m)
    nlat = RR(nlat)
    nfft = RR(nfft)
    kfft = RR(kfft)
    nenu = RR(nenu)
    n = nlat + nfft + nenu
    beta0=RR(beta0)
    beta1=RR(beta1)
    dlat=RR(dlat)
    avg_dlsc=RR(avg_dlsc)
    sdv_dlsc=RR(sdv_dlsc)


    # begin new code
    # compute the bound of i
    bi_inf = RR(0)
    bi_sup = RR(sqrt(beta1)*q/2)
    bi_cur = RR((bi_inf + bi_sup)/2)
    while bi_sup - bi_inf > 0.000000001:
        if B(RR(beta1/RR(2)),RR(RR(RR(2)*pi*dlat*bi_cur)/RR(q))) >= RR(T/N):
            bi_inf = bi_cur
        else :
            bi_sup = bi_cur
        bi_cur = RR((bi_inf + bi_sup)/RR(2))
    bi = bi_sup

    VolLat_lat = RR(-beta1*m/(m+nlat)) * RR(log(RR(q),2)) + RR(beta1*(m+nlat-beta1)) * RR(log(RR(root_Hermite(beta0)), 2))
    VolLat_lsc = RR(- kfft)*RR(log(RR(q), 2)) 
    VolLat = RR(VolLat_lat + VolLat_lsc)

    def sv_D_Sphere(dlsc):
        def func_j(ii):
            ii = RR(ii)
            Bi = B(RR(beta1/RR(2)),RR(RR(RR(2)*pi*dlat*ii)/RR(q)))
        
            # compute j such that int_0^oo psi(dlsc) B(j) ddlsc = T/N/B(i)
            Thres = RR(T/N/Bi)
            j_inf = RR(0)
            j_sup = RR(sqrt(nfft)*q/2)
            j_cur = RR((j_inf + j_sup)/2)
            while j_sup - j_inf > 0.000000001:
                if B(RR(nfft/RR(2))-RR(1),RR(RR(RR(2)*pi*dlsc*j_cur)/RR(q))) >= Thres:              
                    j_inf = j_cur
                else :
                    j_sup = j_cur
                j_cur = RR((j_inf + j_sup)/RR(2))
            return j_sup
        
        def VolWrong(ii):
            VolSphere_i = RR(1) + RR(beta1/2) * RR(log(pi, 2)) + RR(beta1-1)*RR(log(RR(ii), 2)) - RR(log(RR(gamma(RR(beta1/2))), 2))
            jj = RR(func_j(ii))
            VolBall_j = RR(nfft/2) * RR(log(pi, 2)) + RR(nfft)*RR(log(RR(jj), 2)) - RR(log(RR(gamma(RR(nfft/2 + 1))), 2))
            return RR(RR(2)^RR(VolSphere_i + VolBall_j))
        
        res = 0
        step = RR(bi/48.0)
        ii = RR(step/RR(2))
        while ii < bi:
            res += RR(step * VolWrong(ii))
            ii += RR(step)
        #res = RR(numerical_integral(VolWrong, 0, bi, algorithm='qag', max_points=100, eps_abs=1e-20, eps_rel=1e-20)[0])
        return min(0, float(VolLat + log(res, 2)))

    res = RR(0)
    norm = RR(0)
    step = RR((3.0*sdv_dlsc)/25.0)
    dlsc = RR(avg_dlsc-3.0*sdv_dlsc) + RR(step/RR(2))
    while dlsc < avg_dlsc+3.0*sdv_dlsc:
        prob = RR(exp(RR(-0.5 * ((dlsc-avg_dlsc)/sdv_dlsc)^RR(2))) / RR(sdv_dlsc*sqrt(2*pi)))
        norm += RR(prob * step)
        res += RR(step * prob * (RR(2)^RR(sv_D_Sphere(dlsc))))
        dlsc += RR(step)
    return float(log(res/norm, 2))
    # end new code

    '''# begin old code
    def sv_D_sphere(dlsc):
        dlsc = RR(dlsc)

        # compute the bound of i
        bi_inf = RR(0)
        bi_sup = RR(sqrt(beta1)*q/2)
        bi_cur = RR((bi_inf + bi_sup)/2)
        while bi_sup - bi_inf > 0.000000001:
            if B(RR(beta1/RR(2)),RR(RR(RR(2)*pi*dlat*bi_cur)/RR(q))) >= RR(T/N):
                bi_inf = bi_cur
            else :
                bi_sup = bi_cur
            bi_cur = RR((bi_inf + bi_sup)/RR(2))
        bi = bi_inf
        
        # function to maximize
        def func(ii):
            ii = RR(ii)

            # compute j such that G(i,j) = T/N
            Bi = B(RR(beta1/RR(2)),RR(RR(RR(2)*pi*dlat*ii)/RR(q)))
    
            j_inf = RR(0)
            j_sup = RR(sqrt(nfft)*q/2)
            j_cur = RR((j_inf + j_sup)/2)
            while j_sup - j_inf > 0.000000001:
                if Bi * B(RR(nfft/RR(2))-RR(1),RR(RR(RR(2)*pi*dlsc*j_cur)/RR(q))) >= RR(T/N):
                    j_inf = j_cur
                else :
                    j_sup = j_cur
                j_cur = RR((j_inf + j_sup)/RR(2))
            jj = j_inf
            
            # compute P(N_(i,j) > 0)
            VolSphere_i = RR(1) + RR(beta1/2) * RR(log(pi, 2)) + RR(beta1-1)*RR(log(RR(ii), 2)) - RR(log(RR(gamma(RR(beta1/2))), 2))
            VolSphere_j = RR(1) + RR(nfft/2) * RR(log(pi, 2)) + RR(nfft-1)*RR(log(RR(jj), 2)) - RR(log(RR(gamma(RR(nfft/2))), 2))
            VolLat_lat = RR(beta1*nlat/(m+nlat)) * RR(log(RR(q),2)) + RR(beta1*(m+nlat-beta1)) * RR(log(RR(root_Hermite(beta0)), 2))
            VolLat_lsc = RR(nfft - kfft)*RR(log(RR(q), 2))
            li = RR(VolSphere_i + VolLat_lat - beta1*RR(log(RR(q), 2)))
            mj = RR(VolSphere_j + VolLat_lsc - nfft*RR(log(RR(q),2)))
            #mj = RR(log(RR(2) * RR(RR(pi)^RR(nfft/2)) * RR(RR(jj)^RR(nfft - 1)) / RR((RR(q)^RR(kfft)) * RR(gamma(RR(nfft/2)))), 2))
            #li = RR(log(RR(2) * RR(RR(root_Hermite(beta0))^RR(beta1*(m+nlat-beta1))) * RR(RR(pi)^RR(beta1/2)) * RR(RR(ii)^RR(beta1 - 1)) / RR(RR(q)^RR(beta1*m/(m+nlat)) * RR(gamma(RR(beta1/2)))) , 2))
            return float(min(0, mj + li))

        return find_local_maximum(func, 0, bi)[0]

    #return RR(log(numerical_integral(lambda x: RR(exp(RR(-0.5 * ((x-avg_dlsc)/sdv_dlsc)^2)) / RR(sdv_dlsc*sqrt(2*pi))) * RR(2^RR(sv_D_sphere(x))), avg_dlsc - 3*sdv_dlsc, avg_dlsc + 3*sdv_dlsc)[0], 2))
    res = 0
    norm = 0
    step = RR((3.0*sdv_dlsc)/12.0)
    dlsc = RR(avg_dlsc-3.0*sdv_dlsc)
    while dlsc <= avg_dlsc+3.0*sdv_dlsc+0.0000001:
        prob = RR(exp(RR(-0.5 * ((dlsc-avg_dlsc)/sdv_dlsc)^RR(2))) / RR(sdv_dlsc*sqrt(2*pi)))
        norm += RR(prob * step)
        res += RR(step * prob * RR(2)^RR(sv_D_sphere(dlsc)))
        dlsc += RR(step)
    return float(log(res/norm, 2)) 
    # end old code'''

In [22]:

file_name = "optimized_withExperimentalPolar.pkl"

with open(file_name, 'rb') as handle:
		results = pickle.load(handle)

nn = "CC"
sch = Kyber512
parameters = results[sch][nn] # Parameters for the scheme "scheme" with cost model "nn"

#print(parameters)

print("Complexity = ", float(log(parameters['complexity'], 2)))

q = RR(3329)
alpha = (RR(3) if sch == Kyber512 else RR(2))
m = RR(parameters['m'])
beta0 = RR(parameters['beta0'])
beta1 = RR(parameters['beta1'])
nlat = RR(parameters['nlat'])
nfft = RR(parameters['nfft'])
kfft = RR(parameters['kfft'])
nenu = RR(parameters['nenu'])
n = RR(nlat + nfft + nenu)

N = RR(parameters['N'])
Threshold = RR(parameters['treshold'])

dlat = RR(parameters['dlat'])
avg_dlsc = RR(parameters['avg_dlsc'])
sdv_dlsc = RR(parameters['sdv_dlsc'])

eta = RR(parameters['eta'])
false_pos = RR(parameters['epsilon'])

print("q =", int(q))
print("alpha =", int(alpha))
print("n =", int(n))
print("m =", int(m))
print("beta0 =", int(beta0))
print("beta1 =", int(beta1))
print("nenu =", int(nenu))
print("nfft =", int(nfft))
print("kfft =", int(kfft))
print("nlat =", int(nlat))
print("dlat =", dlat.n())
print("avg_dlsc =", avg_dlsc.n())
print("sdv_dlsc =", sdv_dlsc.n())
print("N = ", float(log(N, 2)))
print("T =", float(log(Threshold, 2)))

print("eta = ", float(eta))
print("epsilon temp =", float(log(false_pos, 2)))
print("sqrt(4/3)^beta1 = ", RR(log(sqrt(4/3)^beta1, 2)).n())
R = RR(parameters['R'])
print("R = ", float(log(R, 2)))
print("T_sample = ", float(log(RR(parameters['T_sample']), 2)))
print("N * T_decode = ", float(log(RR(parameters['NT_decode']), 2)))
print("T_fft = ", float(log(RR(parameters['T_FFT']), 2)))

Complexity =  139.53549547971895
q = 3329
alpha = 3
n = 512
m = 475
beta0 = 384
beta1 = 387
nenu = 5
nfft = 52
kfft = 9
nlat = 455
dlat = 2706.85972956168
avg_dlsc = 1528.72926597872
sdv_dlsc = 47.3799888154462
N =  80.30975611045625
T = 43.31060302419016
eta =  0.6582881008232856
epsilon temp = -4.872586415875533
sqrt(4/3)^beta1 =  80.3097561104563
R =  9.390359525563188
T_sample =  139.51463196587108
N * T_decode =  117.707617807604
T_fft =  124.00484693826985


In [24]:
sv_D_ = sv_D(Threshold, N, q, m, alpha, nenu, nlat, nfft, kfft, beta0, beta1, dlat, avg_dlsc, sdv_dlsc)
print("sv_D(T) = ", sv_D_)
sv_N_ = log(RR(RR(RR(1) - RR(erf(RR(RR(Threshold)/RR(sqrt(N))))))/RR(2)), 2)
print("sv_N(T) = ", sv_N_.n())
print("Pwrong = ", max(float(sv_N_), float(sv_D_)))
nb_fp = RR((2^max(sv_N_, sv_D_))*R*(q^kfft))
print("R q^kfft Pwrong = ", log(nb_fp, 2).n())

sv_D(T) =  -160.21273770668532
sv_N(T) =  -119.570804337701
Pwrong =  -119.57080433770109
R q^kfft Pwrong =  -4.87258641587553


Estimation of the smallest vector in the global lattice seen as a random lattice:
$$
\lambda_1\left( q \Lambda\left( \mathbf{B}_{\mathsf{global}}\right)^\vee + \mathbf{r}_{\mathsf{proj}} \right) \approx \frac{q}{ V_{\mathsf{global}}^{\frac{1}{\beta_{\mathsf{sieve}} + n_{\mathsf{fft}}}}}\cdot \sqrt{\tfrac{\beta_{\mathsf{sieve}} + n_{\mathsf{fft}}}{2 \pi e}}
$$
The shortest vector accross $R \cdot q^{k_{\mathsf{fft}}}$ cosets is:
$$
\approx \frac{\lambda_1\left( q \Lambda\left( \mathbf{B}_{\mathsf{global}}\right)^\vee + \mathbf{r}_{\mathsf{proj}} \right)}{\left( R \cdot q^{k_{\mathsf{fft}}} \right)^{\frac{1}{\beta_{\mathsf{sieve}} + n_{\mathsf{fft}}}}}
$$

Length of the target vector:
$$
\approx \sqrt{\tfrac{\alpha (\beta_{\mathsf{sieve}} + n_{\mathsf{fft}})}{2}} 
$$

In [23]:
VolLat_lat = RR(beta1*nlat/(m+nlat)) * RR(log(RR(q),2)) + RR(beta1*(m+nlat-beta1)) * RR(log(RR(root_Hermite(beta0)), 2))
VolLat_lsc = RR(nfft - kfft)*RR(log(RR(q), 2))
V = RR(2^(VolLat_lat+VolLat_lsc))
lambda1 = q * (V^(-1/(beta1+nfft))) * sqrt((beta1 + nfft)/(2*pi*e))
target = sqrt(alpha*(beta1+nfft)/2)
print("target = ", target.n())
print("lambda1 = ", lambda1.n())
print("lambda1/target = ", RR(lambda1/target).n())
r = RR(R*(q^kfft))^RR(1/(beta1+nfft))
# r < lambda1/target ?
print("r = (R q^kfft)^(1/(beta1+nfft)) = ", r.n())
print("lambda1/r = ", RR(lambda1/r).n())

target =  25.6612548407127
lambda1 =  32.6089659168698
lambda1/target =  1.27074712905833
r = (R q^kfft)^(1/(beta1+nfft)) =  1.19853460736472
lambda1/r =  27.2073628216451
